<a href="https://colab.research.google.com/github/juliosdutra/Controle_de_Processos_UFES/blob/main/2.Model-linearization/Lineariza%C3%A7%C3%A3o_usando_Sympy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<left><img src="https://github.com/juliosdutra/Controle_de_Processos_UFES/blob/main/imagens/cabe%C3%A7alho-geral.jpg?raw=true" width="700" height="175">
</left>

**Resolução de parte do exercício nº 3 da Lista 2**

A Questão 3 da Lista 2 trata do descarte de um poluente somente se o efluente apresentar no máximo $5~\mathrm{ppm}$. Para alcançar esse objetivo, um reator agitado é empregado. Esse reator opera com volume, densidade e temperatura constantes, sendo regido pelo seguinte modelo dinâmico:

$$ \frac{dCa}{dt} = D(t)\cdot[Ca_{f}(t)-Ca] - Ra $$

O poluente é alimentado com taxa de diluição D e concentração Caf e tem a seguinte taxa de reação: $$Ra = 15Ca/(1+0.1Ca)~\mathrm{[ppm\cdot h^{-1}]}$$

A condição estacionária é dada por $D_s=4.95~\mathrm{h^{-1}}$, $Ca_{f,s}=10 ~\mathrm{ppm}$ e $Ca_s=3.0~\mathrm{ppm}$.

Nesse contexto, o desafio consiste em determinar o modelo linear usando variáveis-desvio.

O módulo *sympy* é uma biblioteca poderosa para manipulação simbólica e cálculos matemáticos em Python. Aqui está um exemplo de como você pode usar o *sympy* para linearizar o modelo em variáveis-desvio:

In [12]:
import sympy as sp

# Definindo as variáveis simbólicas
D, Caf, Ca = sp.symbols('D Caf Ca')

# Definindo a taxa de reação
Ra = 15 * Ca / (1 + 0.1 * Ca)

# Definindo a função do modelo dinâmico
f_nao_linear = D*(Caf-Ca) - Ra
f_nao_linear

-15*Ca/(0.1*Ca + 1) + D*(-Ca + Caf)

In [13]:
# Dicionário com a condição estacionária (ponto de equilíbrio)
nominal = {Ca: 3, D: 4.95, Caf: 10}

In [14]:
# Derivada de f em relação a Ca
f_nao_linear.diff(Ca).simplify()

-D - 15/(0.1*Ca + 1)**2

In [15]:
# Derivada de f em relação a D
f_nao_linear.diff(D).simplify()

-Ca + Caf

In [16]:
# Derivada de f em relação a Caf
f_nao_linear.diff(Caf).simplify()

D

In [17]:
# Calculando o valor das derivadas no ponto nominal
f_Cas =  f_nao_linear.diff(Ca).subs(nominal)
f_Cafs =  f_nao_linear.diff(Caf).subs(nominal)
f_Ds =  f_nao_linear.diff(D).subs(nominal)

In [18]:
# Expressão de f linearizada
f_linear = f_Cas*Ca + f_Cafs*Caf + f_Ds*D
f_linear

-13.8257396449704*Ca + 4.95*Caf + 7*D

A partir deste ponto, pode-se escrever que

$$ \frac{dCa}{dt}  = -13.826 Ca + 4.95 Caf + 7 D $$

Na forma padrão:

\begin{align*}
&\frac{dCa}{dt} +13.826 Ca =  4.95 Caf + 7 D \\ \\
&\frac{1}{13.826} \frac{dCa}{dt} + Ca =  \frac{4.95}{13.826} Caf + \frac{7}{13.826} D
\end{align*}
<br>
<br>
Finalmente, em termo de variáveis-desvio, o modelo linearizado é:
<br>
<br>
\begin{align*}
&0.072 \frac{dCa}{dt} + Ca = 0.358 Caf + 0.506 D \\ \\
&Ca(0) = 0
\end{align*}

Agora o mesmo exercício será resolvido usando a função "linearize" do pacote *control*.


In [19]:
! pip install -qq control

In [20]:
import control as ct
from control import linearize

In [28]:
def processo(t,x,u, params={}):

    '''modelo de um reator agitado.'''

    # variáveis de estado
    Ca = x

    # variáveis de entrada
    Caf, D = u

    # equação do modelo
    Ra = 15*Ca/(1+0.1*Ca)
    dCa = D*(Caf-Ca) - Ra

    # saida do modelo
    return dCa

In [29]:
# Instanciar o sistema I/O
reator = ct.NonlinearIOSystem(
            processo,
            outfcn = None,
            states = ('Ca'),
            inputs = ('Caf', 'D'),
            outputs = None)

In [30]:
# Variáveis de entrada
Caf0 = 10.0 # Vazão de alimentação
D0 = 4.95 # Substrato na alimentação

# Variáveis de estado
Ca0 = 3.0

# condição inicial (ponto estacionário)
xs = [Ca0]
us = [Caf0, D0]

reator_linear = linearize(reator, xeq=xs, ueq=us)

reator_linear

StateSpace(array([[-13.82573896]]), array([[4.95, 7.  ]]), array([[1.]]), array([[0., 0.]]))

In [31]:
print(reator_linear.A, reator_linear.B)

[[-13.82573896]] [[4.95 7.  ]]


A partir do resultado em espaço de estados, pode-se escrever a equação do modelo linerizado na seguinte forma:

$$\frac{dCa}{dt} = -13.825 Ca + 4.95 Ca_f + 7 D$$

o qual também pode ser colocado na forma padrão como feito na resolução anterior. Em termos de variáveis-desvio, o modelo linear é dado por:

\begin{align*}
  &0.072 \frac{dCa}{dt} + Ca = 0.358 Caf + 0.506 D \\ \\
  &Ca(0) = 0
\end{align*}